# 第四章 构建RAG应用

## 4.1 将LLM 接入 LangChain

LangChain 为基于 LLM 开发自定义应用提供了高效的开发框架，便于开发者迅速地激发 LLM 的强大能力，搭建 LLM 应用。LangChain 也同样支持多种大模型，内置了 OpenAI、LLAMA 等大模型的调用接口。但是，LangChain 并没有内置所有大模型，它通过允许用户自定义 LLM 类型，来提供强大的可扩展性。

In [1]:
from sparkai.core.runnables import RunnableParallel

from zhipuai_llm import ZhipuaiLLM
from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
api_key = os.environ["ZHIPUAI_API_KEY"] #填写控制台中获取的 APIKey 信息

In [2]:
zhipuai_model = ZhipuaiLLM(model_name="glm-4-plus", temperature=0.1, api_key=api_key)

In [3]:
zhipuai_model.invoke("你好，请你自我介绍一下！")

AIMessage(content='你好！我是人工智能助手智谱清言（ChatGLM），是基于智谱 AI 公司于 2024 年训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。', additional_kwargs={}, response_metadata={'time_in_seconds': 1.917}, id='run-7b91b3ac-6af6-44b6-96dc-33b88687e97f-0', usage_metadata={'input_tokens': 13, 'output_tokens': 42, 'total_tokens': 55})

## 4.2 构建检索问答链

在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。

### 4.2.1 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

In [7]:
import sys
sys.path.append("../build_knowledge_database") # 将父目录放入系统路径中

# 使用智谱 Embedding API，注意，需要将上一章实现的封装代码下载到本地
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.vectorstores.chroma import Chroma


加载向量数据库，其中包含了 ../data_base/knowledge_db 下多个文档的 Embedding

In [8]:
# 定义 Embeddings
embedding = ZhipuAIEmbeddings()

# 向量数据库持久化路径
persist_directory = '../data_base/vector_db/chroma'

# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)


In [9]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

向量库中存储的数量：1004


我们可以测试一下加载的向量数据库，我们可以通过`as_retriever`方法把向量数据库构造成检索器。我们使用一个问题 query 进行向量检索。如下代码会在向量数据库中根据相似性进行检索，返回前 k 个最相似的文档。

In [10]:
question = "什么是prompt engineering?"
retriever = vectordb.as_retriever(search_kwargs={"k":3})
docs = retriever.invoke(question)
print(f"检索到的内容数：{len(docs)}")

检索到的内容数：3


打印一下检索到的内容

In [11]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第0个内容: 
 具体来说，首先编写初版 Prompt，然后通过多轮调整逐步改进，直到生成了满意的结果。对于更复杂的应用，可以在多个样本上进行迭代训练，评估 Prompt 的平均表现。在应用较为成熟后，才需要采用在多个样本集上评估 Prompt 性能的方式来进行细致优化。因为这需要较高的计算资源。

总之，Prompt 工程师的核心是掌握 Prompt 的迭代开发和优化技巧，而非一开始就要求100%完美。通过不断调整试错，最终找到可靠适用的 Prompt 形式才是设计 Prompt 的正确方法。

读者可以在 Jupyter Notebook 上，对本章给出的示例进行实践，修改 Prompt 并观察不同输出，以深入理解 Prompt 迭代优化的过程。这会对进一步开发复杂语言模型应用提供很好的实践准备。

三、英文版

产品说明书
-----------------------------------------------------
检索到的第1个内容: 
 第一章 简介

欢迎来到面向开发者的提示工程部分，本部分内容基于吴恩达老师的《Prompt Engineering for Developer》课程进行编写。《Prompt Engineering for Developer》课程是由吴恩达老师与 OpenAI 技术团队成员 Isa Fulford 老师合作授课，Isa 老师曾开发过受欢迎的 ChatGPT 检索插件，并且在教授 LLM （Large Language Model， 大语言模型）技术在产品中的应用方面做出了很大贡献。她还参与编写了教授人们使用 Prompt 的 OpenAI cookbook。我们希望通过本模块的学习，与大家分享使用提示词开发 LLM 应用的最佳实践和技巧。
-----------------------------------------------------
检索到的第2个内容: 
 第二章 提示原则

如何去使用 Prompt，以充分发挥 LLM 的性能？首先我们需要知道设计 Prompt 的原则，它们是每一个开发者设计 Prompt 所必须知道的基础概念。本章讨论了设计高效 Prompt 的两个关键原则：编写清晰、具体的指令和给予模型充足思考时间。掌握这两点，对创建可靠的语言模型交互尤为重要。

首先，Prom

### 4.2.2创建检索链
我们可以使用LangChain的LCEL(LangChain Expression Language, LangChain表达式语言)来构建workflow，LCEL可以支持异步（ainvoke）、流式(stream)、批次处理(batch)等多种运行方式，同时还可以使用LangSmith无缝跟踪。

接下来我们使用刚才定义的retriever定义一个简单的检索链。

In [13]:
from langchain_core.runnables import RunnableLambda
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

combiner = RunnableLambda(combine_docs)
retrieval_chain = retriever | combiner

retrieval_chain.invoke("南瓜书是什么")

'前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；\n• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得\n\n←_←\n\n←_←'

LCEL中要求所有的组成元素都是`Runnable`类型，前面我们见过的`ChatModel`、`PromptTemplate`等都是继承自`Runnable`类。上方的`retrieval_chain`是由检索器`retriever`及组合器`combiner`组成的，由`|`符号串连，数据从左向右传递，即问题先被`retriever`检索得到检索结果，再被`combiner`进一步处理并输出。

### 4.2.3 创建LLM

In [14]:
from zhipuai_llm import ZhipuaiLLM
from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
api_key = os.environ["ZHIPUAI_API_KEY"] #填写控制台中获取的 APIKey 信息

In [ ]:
llm = ZhipuaiLLM(model_name="glm-4-plus", temperature=0.1, api_key=api_key)
llm.invoke("你好，请你自我介绍一下！")

### 4.2.4 构建检索问答链

In [18]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """
使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。最多使用三句话。
尽量使答案简明扼要。请你在回答的最后说“谢谢你的提问！”。
{context}
问题：{input}
"""
#将template通过 PromptTemplate转位可以在LCEL中使用的类型
prompt = PromptTemplate(template=template)

qa_chain = (
    RunnableParallel({"context": retrieval_chain, "input":RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

在上边代码中我们把刚才定义的检索链当作子链作为`prompt`的`context`，再使用`RunnablePassthrough`存储用户的问题作为`prompt`的`input`。又因为这两个操作是并行的，所以我们使用`RunnableParallel`来将他们并行运行。

**检索问答链效果测试**

In [19]:
question_1 = "什么是南瓜书？"
question_2 = "Prompt Engineering for Developer是谁写的？"

In [20]:
result = qa_chain.invoke(question_1)
print("大模型+知识库后的回答 question_1 的结果：")
print(result)

大模型+知识库后的回答 question_1 的结果：
南瓜书是对周志华老师的《机器学习》（西瓜书）中难理解公式的解析和推导细节补充的笔记，旨在帮助读者更好地理解西瓜书内容。它是西瓜书的辅助材料，适合在遇到推导困难时查阅。谢谢你的提问！


In [21]:
result = qa_chain.invoke(question_2)
print("大模型+知识库后的回答 question_2 的结果：")
print(result)

大模型+知识库后的回答 question_2 的结果：
《Prompt Engineering for Developer》课程是由吴恩达老师与 OpenAI 技术团队成员 Isa Fulford 老师合作授课的。谢谢你的提问！


**大模型自己回答的效果**

In [22]:
llm.invoke(question_1).content

'"南瓜书"是对《机器学习》（周志华著，清华大学出版社）一书的一个补充教材的昵称，其正式名称为《机器学习公式详解》。由于这本书的封面是南瓜色的，因此被读者亲切地称为“南瓜书”。\n\n《机器学习》这本书由于其深入浅出的内容和广泛的知识覆盖，被很多学习机器学习的人士作为入门教材。然而，书中涉及大量的数学公式和理论推导，对于一些数学基础不是特别扎实的读者来说可能存在理解上的困难。为了帮助这部分读者更好地理解书中的内容，作者团队推出了这本《机器学习公式详解》，即“南瓜书”。\n\n“南瓜书”对《机器学习》中的公式进行了详细的解释和推导，旨在帮助读者更好地掌握机器学习中的数学原理。这本书的出版受到了广泛的欢迎，成为了机器学习学习者的重要参考资料之一。\n\n需要注意的是，“南瓜书”并不是独立的学习材料，而是作为《机器学习》一书的配套辅导书籍使用的。如果你正在学习机器学习并且使用的是周志华教授的《机器学习》教材，那么“南瓜书”会是一个很好的辅助学习工具。'

In [23]:
llm.invoke(question_2).content

'"Prompt Engineering for Developer" 这个标题看起来像是一篇关于提示工程（Prompt Engineering）的文章或教程，专门针对开发者群体。然而，没有具体的信息，我无法确定它的作者是谁。\n\n提示工程是人工智能领域中的一个新兴概念，特别是在自然语言处理（NLP）和生成模型（如GPT系列）的应用中。它涉及到设计有效的输入提示（prompts）来引导模型生成高质量的输出。\n\n如果你能提供更多的上下文或详细信息，比如文章的发布平台、时间或者其他相关细节，我可能能更好地帮助你找到答案。如果这是一篇在线发布的文章，通常可以在文章的开头或结尾找到作者的信息。'

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

### 4.2.5 向检索链添加聊天记录

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


**传递聊天记录**

在本节中我们将使用 LangChain 中的`ChatPromptTemplate`，即将先前的对话嵌入到语言模型中，使其具有连续对话的能力。 `ChatPromptTemplate`可以接收聊天消息历史记录，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [27]:
from  langchain_core.prompts import ChatPromptTemplate

#问答链的系统prompt
system_prompt = (
    "你是一个问答任务的助手。"
    "请使用检索到的上下文片段回答这个问题。"
    "如果你不知道答案就说不知道。"
    "请使用简洁的话语回答用户。"
    "\n\n"
    "{context}"
)
#制定prompt template
qa_prompt = ChatPromptTemplate(
    [
        ("system", system_prompt),
        ("placeholder", "{chat_history}"),
        ("human","{input}"),
    ]
)

In [28]:
#无历史记录
messages = qa_prompt.invoke(
    {
        "input": "南瓜书是什么？",
        "chat_history": [],
        "context": "",
    }
)
for message in messages.messages:
    print(message.content)

你是一个问答任务的助手。请使用检索到的上下文片段回答这个问题。如果你不知道答案就说不知道。请使用简洁的话语回答用户。


南瓜书是什么？


In [29]:
# 有历史记录
messages = qa_prompt.invoke(
    {
        "input": "你可以介绍一下他吗？",
        "chat_history": [
            ("human", "西瓜书是什么？"),
            ("ai", "西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。"),
        ],
        "context": ""
    }
)
for message in messages.messages:
    print(message.content)

你是一个问答任务的助手。请使用检索到的上下文片段回答这个问题。如果你不知道答案就说不知道。请使用简洁的话语回答用户。


西瓜书是什么？
西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。
你可以介绍一下他吗？


### 4.2.6 带有信息压缩的检索链

因为我们正在搭建的问答链带有支持多轮对话功能，所以与单轮对话的问答链相比会多面临像上方输出结果的问题，即用户最新的对话语义不全，在使用用户问题查询向量数据库时很难检索到相关信息。像上方的“你可以介绍一下他吗？”，其实是“你可以介绍下周志华老师吗？”的意思。为了解决这个问题我们将采取信息压缩的方式，让llm根据历史记录完善用户的问题。

In [30]:
from langchain_core.runnables import RunnableBranch

# 压缩问题的系统 prompt
condense_question_system_template = (
    "请根据聊天记录完善用户最新的问题，"
    "如果用户最新的问题不需要完善则返回用户的问题。"
    )
# 构造 压缩问题的 prompt template
condense_question_prompt = ChatPromptTemplate([
        ("system", condense_question_system_template),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
    ])
# 构造检索文档的链
# RunnableBranch 会根据条件选择要运行的分支
retrieve_docs = RunnableBranch(
    # 分支 1: 若聊天记录中没有 chat_history 则直接使用用户问题查询向量数据库
    (lambda x: not x.get("chat_history", False), (lambda x: x["input"]) | retriever, ),
    # 分支 2 : 若聊天记录中有 chat_history 则先让 llm 根据聊天记录完善问题再查询向量数据库
    condense_question_prompt | llm | StrOutputParser() | retriever,
)

**支持聊天记录的检索问答链**

这里我们使用之前定义的问答模板即`qa_prompt`构造问答链，另外我们通过`RunnablePassthrough.assign`将中间的查询结果存为`"context"`，将最终结果存为`"answer"`。因为查询结果被存为`"context"`，所以我们整合查询结果的函数`combine_docs`也要做相应的改动。

In [31]:
# 重新定义 combine_docs
def combine_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs["context"]) # 将 docs 改为 docs["context"]
# 定义问答链
qa_chain = (
    RunnablePassthrough.assign(context=combine_docs) # 使用 combine_docs 函数整合 qa_prompt 中的 context
    | qa_prompt # 问答模板
    | llm
    | StrOutputParser() # 规定输出的格式为 str
)
# 定义带有历史记录的问答链
qa_history_chain = RunnablePassthrough.assign(
    context = (lambda x: x) | retrieve_docs # 将查询结果存为 content
    ).assign(answer=qa_chain) # 将最终结果存为 answer

**测试检索问答链**

In [32]:
# 不带聊天记录
qa_history_chain.invoke({
    "input": "西瓜书是什么？",
    "chat_history": []
})

{'input': '西瓜书是什么？',
 'chat_history': [],
 'context': [Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 1, 'producer': 'xdvipdfmx (20200315)', 'source': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或者看不懂的公式时再来查阅南瓜书；\n• 对于初学机器学习的小白，西瓜书第1 章和第2 章的公式强烈不建议深究，简单过一下即可，等你学得'),
  Do

In [33]:
# 带聊天记录
qa_history_chain.invoke({
    "input": "南瓜书跟它有什么关系？",
    "chat_history": [
        ("human", "西瓜书是什么？"),
        ("ai", "西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。"),
    ]
})

{'input': '南瓜书跟它有什么关系？',
 'chat_history': [('human', '西瓜书是什么？'),
  ('ai', '西瓜书是指周志华老师的《机器学习》一书，是机器学习领域的经典入门教材之一。')],
 'context': [Document(metadata={'author': '', 'creationDate': "D:20230303170709-00'00'", 'creator': 'LaTeX with hyperref', 'file_path': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': '', 'page': 1, 'producer': 'xdvipdfmx (20200315)', 'source': '../data_base/knowledge_db/pumkin_book/pumpkin_book.pdf', 'subject': '', 'title': '', 'total_pages': 196, 'trapped': ''}, page_content='前言\n“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读\n者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推\n导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充\n具体的推导细节。”\n读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周\n老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书\n中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 本南瓜书只能算是我\n等数学渣渣在自学的时候记下来的笔记，希望能够帮助大家都成为一名合格的“理工科数学基础扎实点的大二\n下学生”。\n使用说明\n• 南瓜书的所有内容都是以西瓜书的内容为前置知识进行表述的，所以南瓜书的最佳使用方法是以西瓜书\n为主线，遇到自己推导不出来或

## 4.3 部署知识库助手


Streamlit 是一种快速便捷的方法，可以直接在 **Python 中通过友好的 Web 界面演示机器学习模型**。在本课程中，我们将学习*如何使用它为生成式人工智能应用程序构建用户界面*。在构建了机器学习模型后，如果你想构建一个 demo 给其他人看，也许是为了获得反馈并推动系统的改进，或者只是因为你觉得这个系统很酷，所以想演示一下：Streamlit 可以让您通过 Python 接口程序快速实现这一目标，而无需编写任何前端、网页或 JavaScript 代码。


### 4.3.1 Streamlit 简介


`Streamlit` 是一个用于快速创建数据应用程序的开源 Python 库。它的设计目标是让数据科学家能够轻松地将数据分析和机器学习模型转化为具有交互性的 Web 应用程序，而无需深入了解 Web 开发。和常规 Web 框架，如 Flask/Django 的不同之处在于，它不需要你去编写任何客户端代码（HTML/CSS/JS），只需要编写普通的 Python 模块，就可以在很短的时间内创建美观并具备高度交互性的界面，从而快速生成数据分析或者机器学习的结果；另一方面，和那些只能通过拖拽生成的工具也不同的是，你仍然具有对代码的完整控制权。

Streamlit 提供了一组简单而强大的基础模块，用于构建数据应用程序：

- st.write()：这是最基本的模块之一，用于在应用程序中呈现文本、图像、表格等内容。

- st.title()、st.header()、st.subheader()：这些模块用于添加标题、子标题和分组标题，以组织应用程序的布局。

- st.text()、st.markdown()：用于添加文本内容，支持 Markdown 语法。

- st.image()：用于添加图像到应用程序中。

- st.dataframe()：用于呈现 Pandas 数据框。

- st.table()：用于呈现简单的数据表格。

- st.pyplot()、st.altair_chart()、st.plotly_chart()：用于呈现 Matplotlib、Altair 或 Plotly 绘制的图表。

- st.selectbox()、st.multiselect()、st.slider()、st.text_input()：用于添加交互式小部件，允许用户在应用程序中进行选择、输入或滑动操作。

- st.button()、st.checkbox()、st.radio()：用于添加按钮、复选框和单选按钮，以触发特定的操作。

这些基础模块使得通过 Streamlit 能够轻松地构建交互式数据应用程序，并且在使用时可以根据需要进行组合和定制，更多内容请查看[官方文档](https://docs.streamlit.io/get-started)